In [123]:
import numpy as np

In [124]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from tensorflow.keras.layers import Conv2D,MaxPooling2D

In [125]:
img_rows,img_cols = 48,48
batch_size = 32

In [126]:
train_data_dir = './dataset/train/'
validation_data_dir = './dataset/validation/'

In [127]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, shear_range=0.3,zoom_range=0.4, width_shift_range=0.4, height_shift_range=0.4, horizontal_flip=True, fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)

In [128]:
train_generator = train_datagen.flow_from_directory(train_data_dir, color_mode='grayscale', target_size=(img_rows,img_cols), batch_size=batch_size,class_mode='categorical', shuffle=True)
validatiion_generator = validation_datagen.flow_from_directory(validation_data_dir, color_mode='grayscale', target_size=(img_rows,img_cols), batch_size=batch_size, class_mode='categorical', shuffle=True)

Found 24282 images belonging to 5 classes.
Found 5937 images belonging to 5 classes.


In [129]:
nb_train_samples = train_generator.n
nb_validation_samples = validatiion_generator.n
num_classes = train_generator.num_classes
# class_weights = [ {idx:(1/val)*nb_train_samples} for idx,val in enumerate(np.bincount(train_generator.classes))]
class_weights = dict(list(enumerate(nb_train_samples/num_classes*np.bincount(train_generator.classes))))
print(class_weights)

{0: 2.0270473328324567, 1: 1.1298157453936348, 2: 1.6246487354476116, 3: 1.6391251518833536, 4: 2.5254290171606866}


In [130]:
np.bincount(train_generator.classes)

array([3993, 7164, 4982, 4938, 3205], dtype=int64)

In [131]:
model = Sequential()

In [132]:
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [133]:
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [134]:
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [135]:
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [136]:
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [137]:
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [138]:
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [139]:
print(model.summary)

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x00000276DC8A5070>>


In [140]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [141]:
checkpoint = ModelCheckpoint('Emotion_little_vgg.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3,verbose=1,restore_best_weights=True)
redure_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3,verbose=1,min_delta=0.0001)
callbacks = [earlystop, checkpoint, redure_lr]

In [142]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [143]:
nb_train_samples = 12102
nb_validation_samples = 2964
epochs = 25

In [144]:
fit = model.fit(train_generator,steps_per_epoch=nb_train_samples//batch_size,epochs=epochs,callbacks=callbacks,validation_data=validatiion_generator,validation_steps=nb_validation_samples//batch_size, class_weight=class_weights)

Epoch 1/25
378/378 [==============================] - 166s 434ms/step - loss: 3.5169 - accuracy: 0.2068 - val_loss: 1.6890 - val_accuracy: 0.2357

Epoch 00001: val_loss improved from inf to 1.68902, saving model to Emotion_little_vgg.h5
Epoch 2/25
378/378 [==============================] - 201s 532ms/step - loss: 2.8703 - accuracy: 0.2204 - val_loss: 1.6110 - val_accuracy: 0.2357

Epoch 00002: val_loss improved from 1.68902 to 1.61102, saving model to Emotion_little_vgg.h5
Epoch 3/25
 87/378 [=====>........................] - ETA: 2:45 - loss: 2.7590 - accuracy: 0.2117

KeyboardInterrupt: 